In [ ]:
import pandas as pd
import os
import subprocess
import re

In [ ]:
def get_columns_to_pause(df):
    pattern = re.compile(r'^pause\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [ ]:
def create_silent_audio(duration, output_file):
    # Command to create silent audio with ffmpeg
    command = [
        'ffmpeg',
        '-f', 'lavfi',
        '-i', 'anullsrc=channel_layout=stereo:sample_rate=48000',
        '-t', duration,
        '-c:a', 'aac',
        '-b:a', '191k',
        output_file
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Silent audio file {output_file} created successfully.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error creating silent audio file: {e}")

In [ ]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'libx264', # needs to stay libx264
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [ ]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [ ]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get file and clean it

In [ ]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)             

In [ ]:
video_schnitt_df

In [ ]:
# for loop trough columns and rows
#   get timestamp pause1_bei
#   create standbild
#   get timestamp pause1_duration
#   create silent audio in right duration
#   merge audio with standbild
# add pause into the video before cut and keep.

In [ ]:
columns_to_pause = get_columns_to_pause(video_schnitt_df)


In [ ]:
# get stanbilder
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "dauer" in col:
            continue
        output = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        timestamp = video_schnitt_df[col][row]

        try:
            print(video_name, "standbild bei:", timestamp, "output:", output)
            get_standbild(video_name,output,timestamp)
        except Exception as e:
            print(e)
        
        counter += 1

In [ ]:
# get audios with silence for duration
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        output = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        timestamp = video_schnitt_df[col][row]
        
        try:
            print(video_name, "silence for:", timestamp, "output:", output)
            create_silent_audio(timestamp,output)
        except Exception as e:
            print(e)

        counter += 1

In [ ]:
# merge all the files
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        video_file = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        audio_file = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        output_file = f"{video_name.split('.')[0]}_merged{counter}.mp4"
        try :
            print(video_name,audio_file,output_file)
            merge_audio_and_video(video_file,audio_file,output_file)
        except Exception as e:
            print(e)
            
        counter += 1